In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from datetime import datetime
import time

In [2]:
os.chdir('C:\\DynamicData')

product = pd.read_csv('product.csv', encoding='utf-8')
session = pd.read_csv('Session.csv', encoding='utf-8')
search1 = pd.read_csv('Search1.csv', encoding='utf-8')
search2 = pd.read_csv('Search2.csv', encoding='utf-8')
custom =  pd.read_csv('custom.csv', encoding='utf-8')
master =  pd.read_csv('Master.csv', encoding='utf-8')

C:\Users\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(product.info()); print()
print(session.info()); print()
print(search1.info()); print()
print(search2.info()); print()
print(custom.info()); print()
print(master.info()); print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5024906 entries, 0 to 5024905
Data columns (total 8 columns):
CLNT_ID      int64
SESS_ID      int64
HITS_SEQ     int64
PD_C         int64
PD_ADD_NM    object
PD_BRA_NM    object
PD_BUY_AM    object
PD_BUY_CT    object
dtypes: int64(4), object(4)
memory usage: 306.7+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712907 entries, 0 to 2712906
Data columns (total 9 columns):
CLNT_ID            int64
SESS_ID            int64
SESS_SEQ           int64
SESS_DT            int64
TOT_PAG_VIEW_CT    float64
TOT_SESS_HR_V      object
DVC_CTG_NM         object
ZON_NM             object
CITY_NM            object
dtypes: float64(1), int64(4), object(4)
memory usage: 186.3+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2884943 entries, 0 to 2884942
Data columns (total 4 columns):
CLNT_ID       int64
SESS_ID       int64
KWD_NM        object
SEARCH_CNT    int64
dtypes: int64(3), object(1)
memory usage: 88.0+ MB
None

<class 'pan

### product 전처리

In [23]:
product.head(3)

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
2,4142395,6158159,85,179538,(not set),[아베다],"39,000",1


In [4]:
product['PD_BUY_AM'] = list(map(lambda x:re.sub("\,", "",x), product['PD_BUY_AM']))
product['PD_BUY_AM'] = product['PD_BUY_AM'].astype('int64')

In [5]:
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype('str')
product['PD_BUY_CT'] = list(map(lambda x:re.sub("\,", "",x), product['PD_BUY_CT']))
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype('int64')

In [32]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5024906 entries, 0 to 5024905
Data columns (total 8 columns):
CLNT_ID      int64
SESS_ID      int64
HITS_SEQ     int64
PD_C         int64
PD_ADD_NM    object
PD_BRA_NM    object
PD_BUY_AM    int64
PD_BUY_CT    int64
dtypes: int64(6), object(2)
memory usage: 306.7+ MB


### session 전처리

In [180]:
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712907 entries, 0 to 2712906
Data columns (total 9 columns):
CLNT_ID            int64
SESS_ID            int64
SESS_SEQ           int64
SESS_DT            int64
TOT_PAG_VIEW_CT    float64
TOT_SESS_HR_V      object
DVC_CTG_NM         object
ZON_NM             object
CITY_NM            object
dtypes: float64(1), int64(4), object(4)
memory usage: 186.3+ MB


In [6]:
#Session 머물지도 않고 바로 나가는 CLIENT들 있다.
session.loc[session['TOT_SESS_HR_V'].isna(),:]

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
38,5878756,7465325,5,20180529,1.0,NaN,mobile,Chungcheongnam-do,Cheonan-si
64,6065738,9564520,26,20180424,1.0,NaN,mobile,Gyeonggi-do,Gwangmyeong-si
65,6065738,9564529,25,20180424,1.0,NaN,mobile,Gyeonggi-do,Gwangmyeong-si
246,6003295,1035552,2,20180913,1.0,NaN,desktop,Daejeon,Daejeon
604,5946142,9600252,4,20180423,1.0,NaN,mobile,Seoul,Seoul
1435,6163287,5612765,98,20180628,1.0,NaN,mobile,Seoul,Seoul
2006,5958856,9540083,20,20180424,1.0,NaN,mobile,Seoul,Seoul
2190,5970505,6643638,3,20180611,1.0,NaN,desktop,Seoul,Seoul
2216,5966034,9201368,8,20180430,1.0,NaN,mobile,Seoul,Seoul
2745,6527707,8400072,29,20180513,1.0,NaN,mobile,Gyeonggi-do,Gwangju-si


In [7]:
#확인
session.iloc[38,]
#이런 사람들은 보면 실제로도 TOT_PAG_VIEW_CT가 1이거나 빈값.

CLNT_ID                      5878756
SESS_ID                      7465325
SESS_SEQ                           5
SESS_DT                     20180529
TOT_PAG_VIEW_CT                    1
TOT_SESS_HR_V                    NaN
DVC_CTG_NM                    mobile
ZON_NM             Chungcheongnam-do
CITY_NM                   Cheonan-si
Name: 38, dtype: object

In [239]:
#추후에 이 이용자들의 행동 양태를 보기 위해 머물지도 않고 바로 나가는 CLIENT들을 따로 저장 (With Session ID)
Zero_Sess_HR_CLNT_ID = session.loc[session['TOT_SESS_HR_V'].isna(),:]['CLNT_ID']
Zero_Sess_HR_SESS_ID = session.loc[session['TOT_SESS_HR_V'].isna(),:]['SESS_ID']
#이 CLIENT들을 제거할 것인가를 결정한 후에 int64형으로 데이터를 바꾸어줄 것
#예를 들어, 빈 칸에 0을 넣을 것인가, 아니면 이 CLIENT들에 해당하는 observation을 제거할 것인가를 결정

In [8]:
session['Timestamp']=0

In [9]:
session['SESS_DT'] = session['SESS_DT'].astype('str')
session['SESS_DT'] = list(map(lambda x:datetime.strptime(x,'%Y%m%d'), session['SESS_DT']))
session['Timestamp'] = list(map(lambda x:datetime.timestamp(x), session['SESS_DT']))
#나중에 Timestamp는 지워줄 것. 
#다만 시간 자료형이 datetime, datetime64, Timestamp가 있어서 부득이하게 한 가지로 맞추어주어야만 했음.

In [10]:
session

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,Timestamp
0,5873599,8641867,9,2018-05-09,82.0,"1,890",mobile,Daejeon,Daejeon,1.525792e+09
1,5873599,6616320,21,2018-06-11,105.0,"1,604",mobile,Busan,Busan,1.528643e+09
2,5873599,5886172,40,2018-06-24,41.0,632,mobile,Daejeon,Daejeon,1.529766e+09
3,5873884,1050889,15,2018-09-13,160.0,"1,035",mobile,Gyeonggi-do,Anyang,1.536764e+09
4,5874461,10298270,5,2018-04-12,13.0,298,mobile,Seoul,Seoul,1.523459e+09
5,5874701,608577,65,2018-09-20,41.0,444,mobile,Seoul,Seoul,1.537369e+09
6,5874829,1666766,87,2018-09-03,36.0,340,mobile,Seoul,Seoul,1.535900e+09
7,5874829,1670494,86,2018-09-03,103.0,976,mobile,Seoul,Seoul,1.535900e+09
8,5874829,1694242,85,2018-09-02,154.0,"3,099",mobile,Seoul,Seoul,1.535814e+09
9,5874829,1559438,92,2018-09-04,24.0,317,mobile,Seoul,Seoul,1.535987e+09


In [141]:
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712907 entries, 0 to 2712906
Data columns (total 9 columns):
CLNT_ID            int64
SESS_ID            int64
SESS_SEQ           int64
SESS_DT            datetime64[ns]
TOT_PAG_VIEW_CT    float64
TOT_SESS_HR_V      object
DVC_CTG_NM         object
ZON_NM             object
CITY_NM            object
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 186.3+ MB


In [18]:
min(session.SESS_DT)

Timestamp('2018-04-01 00:00:00')

In [19]:
max(session.SESS_DT)

Timestamp('2018-09-30 00:00:00')

In [11]:
Sat = pd.date_range(min(session.SESS_DT), max(session.SESS_DT), freq='W-SAT') #Saturday
Sun = pd.date_range(min(session.SESS_DT), max(session.SESS_DT), freq='W-SUN') #Sunday

In [12]:
Sat

DatetimeIndex(['2018-04-07', '2018-04-14', '2018-04-21', '2018-04-28',
               '2018-05-05', '2018-05-12', '2018-05-19', '2018-05-26',
               '2018-06-02', '2018-06-09', '2018-06-16', '2018-06-23',
               '2018-06-30', '2018-07-07', '2018-07-14', '2018-07-21',
               '2018-07-28', '2018-08-04', '2018-08-11', '2018-08-18',
               '2018-08-25', '2018-09-01', '2018-09-08', '2018-09-15',
               '2018-09-22', '2018-09-29'],
              dtype='datetime64[ns]', freq='W-SAT')

In [13]:
Sat_Timestamp=list(map(lambda x:datetime.timestamp(x), Sat))
Sun_Timestamp=list(map(lambda x:datetime.timestamp(x), Sun))

In [14]:
Sat_Timestamp

[1523026800.0,
 1523631600.0,
 1524236400.0,
 1524841200.0,
 1525446000.0,
 1526050800.0,
 1526655600.0,
 1527260400.0,
 1527865200.0,
 1528470000.0,
 1529074800.0,
 1529679600.0,
 1530284400.0,
 1530889200.0,
 1531494000.0,
 1532098800.0,
 1532703600.0,
 1533308400.0,
 1533913200.0,
 1534518000.0,
 1535122800.0,
 1535727600.0,
 1536332400.0,
 1536937200.0,
 1537542000.0,
 1538146800.0]

In [15]:
Holiday = np.array(['20180505', '20180522', '20180606', '20180815', '20180923', '20180924', '20180925'])
Holiday = list(map(lambda x:datetime.strptime(x,'%Y%m%d'), Holiday))

In [16]:
Holiday_Timestamp=list(map(lambda x:datetime.timestamp(x), Holiday))
Holiday_Timestamp

[1525446000.0,
 1526914800.0,
 1528210800.0,
 1534258800.0,
 1537628400.0,
 1537714800.0,
 1537801200.0]

In [17]:
session['SESS_DT']

0         2018-05-09
1         2018-06-11
2         2018-06-24
3         2018-09-13
4         2018-04-12
5         2018-09-20
6         2018-09-03
7         2018-09-03
8         2018-09-02
9         2018-09-04
10        2018-04-13
11        2018-09-30
12        2018-09-25
13        2018-09-23
14        2018-09-20
15        2018-08-25
16        2018-07-16
17        2018-06-06
18        2018-09-05
19        2018-09-27
20        2018-05-11
21        2018-06-03
22        2018-05-21
23        2018-07-27
24        2018-07-31
25        2018-06-09
26        2018-05-10
27        2018-08-07
28        2018-08-22
29        2018-09-23
             ...    
2712877   2018-04-28
2712878   2018-09-16
2712879   2018-05-18
2712880   2018-05-10
2712881   2018-06-27
2712882   2018-06-08
2712883   2018-05-19
2712884   2018-05-13
2712885   2018-05-19
2712886   2018-05-26
2712887   2018-06-15
2712888   2018-06-02
2712889   2018-06-28
2712890   2018-08-24
2712891   2018-07-16
2712892   2018-04-15
2712893   201

In [18]:
session['Rest']=0
session['Timestamp'].head(4)

0    1.525792e+09
1    1.528643e+09
2    1.529766e+09
3    1.536764e+09
Name: Timestamp, dtype: float64

In [19]:
All_Timestamp = Sat_Timestamp + Sun_Timestamp + Holiday_Timestamp
All_Timestamp

[1523026800.0,
 1523631600.0,
 1524236400.0,
 1524841200.0,
 1525446000.0,
 1526050800.0,
 1526655600.0,
 1527260400.0,
 1527865200.0,
 1528470000.0,
 1529074800.0,
 1529679600.0,
 1530284400.0,
 1530889200.0,
 1531494000.0,
 1532098800.0,
 1532703600.0,
 1533308400.0,
 1533913200.0,
 1534518000.0,
 1535122800.0,
 1535727600.0,
 1536332400.0,
 1536937200.0,
 1537542000.0,
 1538146800.0,
 1522508400.0,
 1523113200.0,
 1523718000.0,
 1524322800.0,
 1524927600.0,
 1525532400.0,
 1526137200.0,
 1526742000.0,
 1527346800.0,
 1527951600.0,
 1528556400.0,
 1529161200.0,
 1529766000.0,
 1530370800.0,
 1530975600.0,
 1531580400.0,
 1532185200.0,
 1532790000.0,
 1533394800.0,
 1533999600.0,
 1534604400.0,
 1535209200.0,
 1535814000.0,
 1536418800.0,
 1537023600.0,
 1537628400.0,
 1538233200.0,
 1525446000.0,
 1526914800.0,
 1528210800.0,
 1534258800.0,
 1537628400.0,
 1537714800.0,
 1537801200.0]

In [28]:
session.ix[session['Timestamp'].isin(All_Timestamp),'Rest']=1

C:\Users\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [33]:
session

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,Timestamp,Rest
0,5873599,8641867,9,2018-05-09,82.0,"1,890",mobile,Daejeon,Daejeon,1.525792e+09,0
1,5873599,6616320,21,2018-06-11,105.0,"1,604",mobile,Busan,Busan,1.528643e+09,0
2,5873599,5886172,40,2018-06-24,41.0,632,mobile,Daejeon,Daejeon,1.529766e+09,1
3,5873884,1050889,15,2018-09-13,160.0,"1,035",mobile,Gyeonggi-do,Anyang,1.536764e+09,0
4,5874461,10298270,5,2018-04-12,13.0,298,mobile,Seoul,Seoul,1.523459e+09,0
5,5874701,608577,65,2018-09-20,41.0,444,mobile,Seoul,Seoul,1.537369e+09,0
6,5874829,1666766,87,2018-09-03,36.0,340,mobile,Seoul,Seoul,1.535900e+09,0
7,5874829,1670494,86,2018-09-03,103.0,976,mobile,Seoul,Seoul,1.535900e+09,0
8,5874829,1694242,85,2018-09-02,154.0,"3,099",mobile,Seoul,Seoul,1.535814e+09,1
9,5874829,1559438,92,2018-09-04,24.0,317,mobile,Seoul,Seoul,1.535987e+09,0


In [36]:
session.drop('Timestamp', axis=1)

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,Rest
0,5873599,8641867,9,2018-05-09,82.0,"1,890",mobile,Daejeon,Daejeon,0
1,5873599,6616320,21,2018-06-11,105.0,"1,604",mobile,Busan,Busan,0
2,5873599,5886172,40,2018-06-24,41.0,632,mobile,Daejeon,Daejeon,1
3,5873884,1050889,15,2018-09-13,160.0,"1,035",mobile,Gyeonggi-do,Anyang,0
4,5874461,10298270,5,2018-04-12,13.0,298,mobile,Seoul,Seoul,0
5,5874701,608577,65,2018-09-20,41.0,444,mobile,Seoul,Seoul,0
6,5874829,1666766,87,2018-09-03,36.0,340,mobile,Seoul,Seoul,0
7,5874829,1670494,86,2018-09-03,103.0,976,mobile,Seoul,Seoul,0
8,5874829,1694242,85,2018-09-02,154.0,"3,099",mobile,Seoul,Seoul,1
9,5874829,1559438,92,2018-09-04,24.0,317,mobile,Seoul,Seoul,0


### search1 전처리

In [94]:
search1.head(4) #info를 보면 딱히 전처리 필요 없어 보임.

,CLNT_ID,SESS_ID,KWD_NM,SEARCH_CNT
0,5607714,7112876,빌리프 아이크림,6
1,5607714,4090791,프리메라 마스크팩,3
2,5607714,4090791,여성청결제,1
3,5612428,1876482,명품가방,1


### search2 전처리

In [95]:
search2.head(4)

,SESS_DT,KWD_NM,SEARCH_CNT
0,20180407,닥스원피스,8
1,20180407,닥터지 브라이트닝,1
2,20180407,달팡 인트랄,2
3,20180407,대상트,1


In [81]:
search2['SEARCH_CNT'] = search2['SEARCH_CNT'].astype('str')
search2['SEARCH_CNT'] = list(map(lambda x:re.sub("\,", "",x), search2['SEARCH_CNT']))
search2['SEARCH_CNT'] = search2['SEARCH_CNT'].astype('int32')

In [82]:
search2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8051172 entries, 0 to 8051171
Data columns (total 3 columns):
SESS_DT       int64
KWD_NM        object
SEARCH_CNT    int32
dtypes: int32(1), int64(1), object(1)
memory usage: 153.6+ MB


In [83]:
custom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671679 entries, 0 to 671678
Data columns (total 3 columns):
CLNT_ID        671679 non-null int64
CLNT_GENDER    671679 non-null object
CLNT_AGE       671679 non-null int64
dtypes: int64(2), object(1)
memory usage: 15.4+ MB


In [96]:
custom.head(4)

,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,4830726,F,30
1,4830874,F,40
2,4830975,F,30
3,4831275,F,30


In [97]:
master.head(4)

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
1,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말
2,61729,88A2933253배트맨스웨트티 - 블랙 / 130,유아동의류,유아의류상의,영유아티셔츠/탑
3,61537,"닥터마틴 아드리안 블랙, 체리레드 - 02_체리레드 / 250mm(6)",패션잡화,남성화,남성부츠
